In [1]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import numpy as np
np.random.seed(10)

import re
import os

def rm_tags(text):
        re_tag = re.compile(r'<[^>]+>')
        return re_tag.sub('', text)
    
def read_files(filetype):
    path = 'aclImdb/'
    file_list = []
    
    positive_path = path + filetype + '/pos/'
    for f in os.listdir(positive_path):
        file_list += [positive_path+f]
        
    negative_path = path + filetype + '/neg/'
    for f in os.listdir(negative_path):
        file_list += [negative_path+f]
        
    print('read', filetype, 'files:', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    all_texts = []
    
    for fi in file_list:
        with open(fi, encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
            
    return all_labels, all_texts

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_y, train_text = read_files('train')
test_y, test_text = read_files('test')

predict_size = 'small'
if predict_size == 'small':
    num_words = 2000
    words_len = 100
else:
    num_words = 3800
    words_len = 380
#token = Tokenizer(num_words=2000)
token = Tokenizer(num_words=num_words)
token.fit_on_texts(train_text)

read train files: 25000
read test files: 25000


In [7]:
print(token.document_count)

25000


In [8]:
print(token.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'as': 12, 'with': 13, 'for': 14, 'was': 15, 'but': 16, 'film': 17, 'movie': 18, 'his': 19, 'on': 20, 'are': 21, 'he': 22, 'you': 23, 'not': 24, 'one': 25, 'have': 26, 'be': 27, 'by': 28, 'all': 29, 'an': 30, 'at': 31, 'who': 32, 'from': 33, 'her': 34, 'they': 35, 'has': 36, 'so': 37, 'like': 38, "it's": 39, 'about': 40, 'very': 41, 'out': 42, 'or': 43, 'good': 44, 'more': 45, 'when': 46, 'some': 47, 'what': 48, 'if': 49, 'just': 50, 'she': 51, 'there': 52, 'story': 53, 'time': 54, 'my': 55, 'great': 56, 'well': 57, 'up': 58, 'which': 59, 'their': 60, 'see': 61, 'can': 62, 'also': 63, 'really': 64, 'would': 65, 'will': 66, 'me': 67, 'had': 68, 'only': 69, 'we': 70, 'him': 71, 'even': 72, 'most': 73, 'other': 74, 'were': 75, 'first': 76, 'than': 77, 'much': 78, 'no': 79, 'its': 80, 'into': 81, 'people': 82, 'best': 83, 'love': 84, 'get': 85, 'how': 86, 'been': 87, 'life': 88, 'becaus

In [3]:
train_text_seq = token.texts_to_sequences(train_text)
test_text_seq = token.texts_to_sequences(test_text)

In [13]:
print(train_text[0])
print(len(train_text[0]))

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
806


In [14]:
print(train_text_seq[0])
print(len(train_text_seq[0]))

[300, 6, 3, 1194, 190, 8, 31, 1, 168, 54, 12, 47, 74, 40, 412, 88, 127, 12, 55, 122, 7, 1, 496, 67, 5, 303, 11, 1995, 6, 78, 5, 518, 77, 6, 1, 5, 1864, 1, 1736, 32, 62, 61, 197, 141, 60, 1, 4, 1, 251, 800, 29, 67, 4, 1, 9, 676, 2, 60, 1736, 46, 9, 195, 1, 317, 7, 59, 3, 1653, 1158, 5, 192, 1, 412, 9, 1199, 31, 300, 3, 283, 375, 180, 126, 5, 25, 4, 135, 1653, 5, 300, 9, 536, 11, 96, 1334, 4, 55, 456, 98, 11, 300, 6, 244, 48, 3, 11, 8, 248]
107


In [4]:
x_train = sequence.pad_sequences(train_text_seq, maxlen=words_len)
x_test = sequence.pad_sequences(test_text_seq, maxlen=words_len)

In [16]:
print(len(x_train[0]))

100


In [18]:
print(len(test_text_seq[1]))
print(len(x_test[0]))

290
100


In [5]:
# building the keras layouts.

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(output_dim=32, input_dim=num_words, input_length=words_len))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.35))

# the output
model.add(Dense(units=1, activation='sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               819456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 883,713
Trainable params: 883,713
Non-trainable params: 0
_________________________________________________________________


In [7]:
# model training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history = model.fit(x_train, train_y, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 13s - loss: 0.4744 - acc: 0.7575 - val_loss: 0.5163 - val_acc: 0.7572
Epoch 2/10
 - 11s - loss: 0.2691 - acc: 0.8898 - val_loss: 0.4737 - val_acc: 0.7948
Epoch 3/10
 - 11s - loss: 0.1637 - acc: 0.9383 - val_loss: 0.5853 - val_acc: 0.7726
Epoch 4/10
 - 11s - loss: 0.0815 - acc: 0.9726 - val_loss: 0.9371 - val_acc: 0.7242
Epoch 5/10
 - 11s - loss: 0.0536 - acc: 0.9813 - val_loss: 0.9779 - val_acc: 0.7548
Epoch 6/10
 - 11s - loss: 0.0389 - acc: 0.9866 - val_loss: 0.9038 - val_acc: 0.7874
Epoch 7/10
 - 11s - loss: 0.0280 - acc: 0.9905 - val_loss: 1.1116 - val_acc: 0.7654
Epoch 8/10
 - 11s - loss: 0.0265 - acc: 0.9910 - val_loss: 1.2548 - val_acc: 0.7526
Epoch 9/10
 - 10s - loss: 0.0243 - acc: 0.9913 - val_loss: 1.1360 - val_acc: 0.7776
Epoch 10/10
 - 11s - loss: 0.0246 - acc: 0.9909 - val_loss: 1.5693 - val_acc: 0.7136


In [8]:
# evaluation
scores = model.evaluate(x_test, test_y, verbose=1)
scores

25000/25000 [==============================] - 3s 110us/step


[0.9850206789955497, 0.80812]

In [13]:
# RNN model building
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

model_rnn = Sequential()
model_rnn.add(Embedding(output_dim=32, input_dim=num_words, input_length=words_len))
model_rnn.add(Dropout(0.35))
model_rnn.add(SimpleRNN(units=16))
model_rnn.add(Dense(units=256, activation='relu'))
model_rnn.add(Dropout(0.35))
model_rnn.add(Dense(units=1, activation='sigmoid'))

In [14]:
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 69,393
Trainable params: 69,393
Non-trainable params: 0
_________________________________________________________________


In [18]:
# model training
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history_rnn = model_rnn.fit(x_train, train_y, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 10s - loss: 0.5401 - acc: 0.7276 - val_loss: 0.6286 - val_acc: 0.6792
Epoch 2/10
 - 10s - loss: 0.3763 - acc: 0.8401 - val_loss: 0.6020 - val_acc: 0.7292
Epoch 3/10
 - 9s - loss: 0.3381 - acc: 0.8587 - val_loss: 0.4867 - val_acc: 0.7954
Epoch 4/10
 - 8s - loss: 0.3094 - acc: 0.8715 - val_loss: 0.6987 - val_acc: 0.6930
Epoch 5/10
 - 8s - loss: 0.2945 - acc: 0.8792 - val_loss: 0.6562 - val_acc: 0.7084
Epoch 6/10
 - 8s - loss: 0.2590 - acc: 0.8962 - val_loss: 0.5637 - val_acc: 0.7704
Epoch 7/10
 - 8s - loss: 0.2375 - acc: 0.9020 - val_loss: 0.5443 - val_acc: 0.7856
Epoch 8/10
 - 8s - loss: 0.2217 - acc: 0.9131 - val_loss: 0.6163 - val_acc: 0.7948
Epoch 9/10
 - 8s - loss: 0.1952 - acc: 0.9242 - val_loss: 0.8172 - val_acc: 0.7328
Epoch 10/10
 - 9s - loss: 0.1880 - acc: 0.9261 - val_loss: 0.7338 - val_acc: 0.7828


In [20]:
scores_rnn = model_rnn.evaluate(x_test, test_y, verbose=1)
scores_rnn

25000/25000 [==============================] - 5s 212us/step


[0.5593941188144684, 0.82188]

In [21]:
# LSTM model building
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

model_lstm = Sequential()
model_lstm.add(Embedding(output_dim=32, input_dim=num_words, input_length=words_len))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=32))
model_lstm.add(Dense(units=256, activation='relu'))
model_lstm.add(Dropout(0.20))
model_lstm.add(Dense(units=1, activation='sigmoid'))

model_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 81,025
Trainable params: 81,025
Non-trainable params: 0
_________________________________________________________________


In [22]:
# model training
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history_lstm = model_lstm.fit(x_train, train_y, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 27s - loss: 0.5043 - acc: 0.7440 - val_loss: 0.4721 - val_acc: 0.7984
Epoch 2/10
 - 24s - loss: 0.3295 - acc: 0.8596 - val_loss: 0.3716 - val_acc: 0.8218
Epoch 3/10
 - 22s - loss: 0.3004 - acc: 0.8754 - val_loss: 0.3888 - val_acc: 0.8194
Epoch 4/10
 - 22s - loss: 0.2869 - acc: 0.8817 - val_loss: 0.5065 - val_acc: 0.7580
Epoch 5/10
 - 23s - loss: 0.2693 - acc: 0.8874 - val_loss: 0.5193 - val_acc: 0.7572
Epoch 6/10
 - 23s - loss: 0.2535 - acc: 0.8982 - val_loss: 0.6974 - val_acc: 0.7154
Epoch 7/10
 - 23s - loss: 0.2425 - acc: 0.9020 - val_loss: 0.5362 - val_acc: 0.7746
Epoch 8/10
 - 23s - loss: 0.2284 - acc: 0.9090 - val_loss: 0.5563 - val_acc: 0.7668
Epoch 9/10
 - 22s - loss: 0.2174 - acc: 0.9135 - val_loss: 0.5850 - val_acc: 0.7842
Epoch 10/10
 - 22s - loss: 0.2109 - acc: 0.9157 - val_loss: 0.5913 - val_acc: 0.7848


In [23]:
scores_lstm = model_lstm.evaluate(x_test, test_y, verbose=1)
scores_lstm

25000/25000 [==============================] - 11s 438us/step


[0.4609196385240555, 0.82868]

In [9]:
# predication
predict = model.predict_classes(x_test)
predict_classes = predict.reshape(-1)

In [10]:
predict_classes[:10]

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [11]:
sentiment_wording = {1: '正評', 0: '負評'}
def show_test_sentiment(i):
    print('----------------------------------------')
    print('the movie text: ')
    print(test_text[i])
    print('the sentiment prediction result: predict: {0}, original: {1}'.format(
        sentiment_wording[test_y[i]], sentiment_wording[predict_classes[i]]))
    print('----------------------------------------')

In [12]:
for i in range(10):
    show_test_sentiment(i)

----------------------------------------
the movie text: 
I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.
the sentiment prediction result: predict: 正評, original: 正評
----------------------------------------
----------------------------------------
the mo

In [18]:
# customized rating

user_review = """Walking out all I could think was "WOW!". This movie is everything the animated version was and so much more!

The live action movie follows the animated movie very closely. It has been a little bit since I saw the animated version, but I didn't notice anything missing from the original. The live action version has a few extra scenes than the animated version which adds to the backstory. I really thought these scenes were an excellent addition.

I really loved this movie and it surpassed my expectations. I was a little worried about a live action movie after always loving the original - I wasn't sure it would measure up. The music was wonderful and the new songs that have been added compliment the original soundtrack perfectly. The costumes are wonderfully colorful and bright and scenes are beautiful. Cinematography and the CGI effects were also very good and felt seamless. 

There are so many things that make movies uniquely Disney and those qualities were evident in this movie too. The "Be Our Guest" scene had me flashing back to Fantasia for instance, and the overall magic of the film in all of it's beauty could really only be Disney. 

I highly recommend this film."""

user_review_seq = token.texts_to_sequences([user_review])

padded_user_review_seq = sequence.pad_sequences(user_review_seq, maxlen=100)


In [25]:
predict_result = model.predict_classes(padded_user_review_seq)
sentiment_wording[predict_result[0][0]]

'正評'

In [29]:
def predict_review(input_text):
    user_review_seq = token.texts_to_sequences([input_text])

    padded_user_review_seq = sequence.pad_sequences(user_review_seq, maxlen=words_len)
    
    predict_result = model.predict_classes(padded_user_review_seq)
    print('----------------------------------------------')
    print('the movie to predict: {0}'.format(input_text))
    
    print('sentiment result: {0}'.format(sentiment_wording[predict_result[0][0]]))
    print('----------------------------------------------')

In [30]:
#original rating= 1
input_text = """
Where do I start. This adaptation of Disney's 1991 Beauty and the Beast was an utter disappointment. Emma Watson as Belle was extremely unconvincing from the start to the end. She had the same expressions as the actress from Twilight. The animators did a terrible job with the Beast. He looked fake and lifeless. They could have used special makeup to create the beast similar to the Grinch where we get to see Jim Carrey's expressions. The side character animations were poorly executed. Overall I felt the film was rushed as there was lack of compassion and chemistry between the characters. There was a lot of CGI and green screen which could have been replaced by normal acting, because then why make an animated version of an animated film? This is by far the worst remake of an animated classic.
"""
predict_review(input_text)

----------------------------------------------
the movie to predict: 
Where do I start. This adaptation of Disney's 1991 Beauty and the Beast was an utter disappointment. Emma Watson as Belle was extremely unconvincing from the start to the end. She had the same expressions as the actress from Twilight. The animators did a terrible job with the Beast. He looked fake and lifeless. They could have used special makeup to create the beast similar to the Grinch where we get to see Jim Carrey's expressions. The side character animations were poorly executed. Overall I felt the film was rushed as there was lack of compassion and chemistry between the characters. There was a lot of CGI and green screen which could have been replaced by normal acting, because then why make an animated version of an animated film? This is by far the worst remake of an animated classic.

sentiment result: 負評
----------------------------------------------


In [31]:
# original rate: 1
input_text = """
Bad movie for sure. Disney movies are most of the times one of the best I have ever seen, but not always this is true which is the case here. This movie was so boring so I was wandering what to do 1 hour and 50 minutes, only the last 10 minutes of the movie is worth seeing. Budget 160 millions dollars and the result is total disappointment. All this musical elements in the movie is just too much, it takes away the real feeling that this movie is supposed to bring to the viewers. Ema Watson clearly is worst choice for this role, she was looking like Beast's daughter when standing next to him, she is just too young for this role. This movie should be rated not to be viewed by viewers over 16 years of age. Overall very bad movie, it is a clear waste of time to see this movie, but we will forget Disney and will wait for them to get back on track this month of May.
"""
predict_review(input_text)

----------------------------------------------
the movie to predict: 
Bad movie for sure. Disney movies are most of the times one of the best I have ever seen, but not always this is true which is the case here. This movie was so boring so I was wandering what to do 1 hour and 50 minutes, only the last 10 minutes of the movie is worth seeing. Budget 160 millions dollars and the result is total disappointment. All this musical elements in the movie is just too much, it takes away the real feeling that this movie is supposed to bring to the viewers. Ema Watson clearly is worst choice for this role, she was looking like Beast's daughter when standing next to him, she is just too young for this role. This movie should be rated not to be viewed by viewers over 16 years of age. Overall very bad movie, it is a clear waste of time to see this movie, but we will forget Disney and will wait for them to get back on track this month of May.

sentiment result: 負評
-----------------------------------

In [32]:
# original rating: 4
input_text = """
Here you have one of Disney's most well loved and most successful masterpieces. To a lot of people, THE most loved. 

So how can you go wrong with this one, right Disney? 

That's what I thought at first. Then they announced the two main leads; Emma Watson as Belle, and Dan Stevens as the Beast. Come again? How on earth could Disney have got this so wrong? Who the heck was in charge of casting?

For the two main stars of the show, Belle and the Beast, you need STRONG leads. Emma Watson just got lucky being cast in Harry Potter. Is she one of the most talented actresses out there? No. Can she give us the same emotion Belle shows that we all grew up loving? No. Can she sing? Apparently not, because all I heard was this terrible auto-tuned voice in every song she sang. And it wasn't subtle either. What the heck Disney! Just because Emma grew up in Paris didn't make her 'the perfect Belle.'

Dan Stevens. I liked him in Downton Abbey. However, I would never in a million years thought to myself; "He'd make a great live-action Beast." Never! And oh gosh, his tacky CGI animation was just as bad as Emma's awful auto tuned voice. A big, CGI goat-man.

Also, age difference much? The kiss that they finally shared at the end was awkward, almost cringy, because Emma Watson looks like a teenage girl, kissing her nearly-middle aged school teacher or something. It was just weird.

How could you screw this up Disney? You had the money! Just, why?! The two main characters did nothing for me. I liked Belle's costumes more than I liked Belle. What does that say? Her clothes had more emotion?

Why was Belle viewed as 'odd' in her village full of weird characters? Bonjour was waaay too over the top. 

Tale As Old As Time was too cockney. It may as well have been Mrs Lovett from Sweeney Todd singing it.

Also, Be Our Guest? How could you screw up Be Our Guest for crying out loud? Well Disney did that too. Great job guys.

Gaston. Is it only me who thought that the Villain stole the show? Because Luke Evens did for me. He was awesome. When they had first cast Luke Evens, I had a lot of doubts, but as I watched him on screen, actually SINGING (he can sing) and dancing and having fun with his role, I was sorry for ever doubting him. He and Josh Gad breathed life into the songs we already loved from the original. I loved their (slightly one sided) bromance.

I was hoping LaFou would be more 'animated' like he was in the original, because I thought Josh Gad was perfect for this role, but I still enjoyed him as a more real, toned down version.

Best things about this movie: The costumes. (I chocked up when I saw that yellow dress.) The score. (NOT the singing.) The cinematography. (Minus all the unnecessary CGI) The choreography. Gaston and LeFou. Lumière's cute relationship with Plumette.

Okay I'm done. It's gonna take at least a week for me to get over this. Maybe I'll watch the original this weekend."""

predict_review(input_text)

----------------------------------------------
the movie to predict: 
Here you have one of Disney's most well loved and most successful masterpieces. To a lot of people, THE most loved. 

So how can you go wrong with this one, right Disney? 

That's what I thought at first. Then they announced the two main leads; Emma Watson as Belle, and Dan Stevens as the Beast. Come again? How on earth could Disney have got this so wrong? Who the heck was in charge of casting?

For the two main stars of the show, Belle and the Beast, you need STRONG leads. Emma Watson just got lucky being cast in Harry Potter. Is she one of the most talented actresses out there? No. Can she give us the same emotion Belle shows that we all grew up loving? No. Can she sing? Apparently not, because all I heard was this terrible auto-tuned voice in every song she sang. And it wasn't subtle either. What the heck Disney! Just because Emma grew up in Paris didn't make her 'the perfect Belle.'

Dan Stevens. I liked him in D

In [50]:
# original rating:  4
input_text = """
I will be the first person to acknowledge that remakes often can be a necessary part of this industry. But this entire project just didn't sit well with me from the get-go. So let's dive in, shall we?

1) VOCALS:

This was a musical, and as such being the case, it is inexcusable to have Watson as the lead for such a classic. She never had vocal coaching, and from the trailers alone this was clearly going to be CPU corrected throughout to compensate.

As predicted, her voice has been largely manipulated in post. And considering that Walt Disney himself made such a big deal out of the musical production in all of his movies, I cannot understand the complete and total neglect of the soundtrack (again, something largely responsible for what made the original so magical).

2) CGI:

Take a moment and go look at some pictures of CGI work in Life of Pi. Then compare this to Beast's face or any other animation in this remake. I would then like you to ask yourself the following:

How could it be that Life of Pi managed to create imagery of far greater quality on a budget which was $40 million LESS than that which Disney was working with.

3) PRODUCTION & SET DESIGN:

I liken this aspect to someone making everything out of cardboard and then painting it in solid colors so as to exactly match the original. Again, I do not understand why the set looked so lifeless considering Disney has opted for a live-action remake. I couldn't help but feel this was lacking any form of originality with which to carry the movie.

4) CASTING:

Lucy Bevan was the casting director for this one. Some of her more noteworthy credits include The Golden Compass, An Education, Nanny McPhee Returns, Maleficent, 300, and Cinderella. It is quite clear after taking a good look at her directorial resume just how hit-or-miss this person is when it comes to choosing the right people for the right roles.

And once again, with such a strong backing for this film, I find myself left dumbfounded wondering why Disney would take such a gamble with arguably the greatest story to ever come out of the magic factory.

ADDITIONAL ANALYSIS:

This entire project just feels like it was overseen by an entirely new group of execs and with no other intent other than the money. I wouldn't be so harsh, but this is Disney. An organization which prides itself on utilizing fantastic music, strong performances, astounding visuals, and detailed CGI. Yet every single one of those pillars has been overlooked.

I have no inherent problems with Watson. She has proved capable in one of my favorites: The Perks of Being a Wallflower (2012). However, this role was not for her. This is one time where taking the risk (aka learning to sing for the part) was a mistake. A task far too great for her to overcome.

If you are in your early teens, have young children, or are an avid fan of Harry Potter, then this Disney remake is for you. The excitement of seeing familiar characters on screen in real life will be excitement enough, bringing about a sense of nostalgia long sought after.

Simply put, this isn't how films were meant to be created. Shame on you Disney. It's not OK for a company of such proportions to ignore standards spanning nearly a century, instead demonstrating such blatant disregard for the original.

I give this 4 stars. There was nothing special about it. The problem was that Disney chose to reboot a film which was already an 8/10 and failed to bring about any hint of originality.

Looking ahead, I really hope you don't ruin Mulan and The Little Mermaid with similar mistakes. We are currently 0/1. I will remain optimistic, however, given the success of Cinderella. But for the love of all that is good, Disney! PLEASE pull it together in upcoming endeavors.

A FINAL NOTE:

For too long I have seen disingenuous reviews scattered throughout in the opening weeks of any larger budget film release. This is really starting to bother me IMDb! You guys should be able to create some sort of list or set of requirements that must be fulfilled in order to leave a review: for example, requiring users to have rated at least 500+ films, contributing to at least 3 different parts of this community, or even just ensuring the account has no suspicious activity which would be indicative of illegitimate, over the top reviews.

There are of course going to be opposing views out there, however I challenge those suspicious of unwarranted criticism like what I have offered here to go and inspect the profiles to see which ratings are associated with long-time active members. For this movie alone, nearly half of all ratings over 8/10 come from accounts which were made just weeks ago and whose activity consists of but one single rating for the film. These falsified reviews are taking away from the validity of overall IMDb ratings!

Please, please, PLEASE do something about this :(

----- 4/10 STARS -------- Review by Searsino -----"""
predict_review(input_text)

ValueError: Error when checking input: expected embedding_4_input to have shape (400,) but got array with shape (100,)